In [19]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
import nltk

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Harsh
[nltk_data]     Clean\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [13]:
data = pd.read_csv('../../Warehouse/Reviews/app_reviews_merged.csv')

In [14]:
# Preprocess the data
data['content'] = data['content'].str.lower()

# Specific features to look for in reviews
features = ['login', 'account', 'free fire', 'transfer', 'teenager']

In [15]:
# Create a dictionary to store user patterns
user_patterns = defaultdict(lambda: {'average_score': 0, 'feature_mentions': Counter()})

# Calculate the average score per user
user_avg_scores = data.groupby('userName')['score'].mean()

In [23]:
# Tokenize the review content and count the mentions of specific features for each user
for _, row in data.iterrows():
    user = row['userName']
    content = row['content']
    tokens = word_tokenize(content)
    
    for feature in features:
        feature_tokens = word_tokenize(feature)
        if set(feature_tokens).issubset(set(tokens)):
            user_patterns[user]['feature_mentions'][feature] += 1 #type: ignore

In [24]:
# Update the average score in user_patterns
for user, avg_score in user_avg_scores.items():
    user_patterns[user]['average_score'] = avg_score

In [25]:
# Group users based on their average scores and feature mentions
high_rating_users = [user for user, pattern in user_patterns.items() if pattern['average_score'] >= 4] #type: ignore
low_rating_users = [user for user, pattern in user_patterns.items() if pattern['average_score'] < 4] #type: ignore

In [26]:
print("High rating users:")
print(high_rating_users)
print("\nLow rating users:")
print(low_rating_users)

High rating users:
['Vishal Yadav', 'Aditya Prasad', 'Nishant Chaudhary', 'Rahul Kumar', 'Nikhil Yadav', 'Suraj Yadav', 'Santosh Chavan', 'Prince Singh', 'Shivam Kumar', 'Krishna Singh', 'Sahil Kumar', 'Piyush Gupta', 'Harsh Verma', 'Aman Yadav', 'Aditya Patel', 'Krishna Sharma', 'Arbaz Khan', 'Abhishek Sharma', 'Raj Kushwaha', 'Neeraj Yadav', 'Kaif Shaikh', 'Vansh Choudhary', 'Smit Patel', 'Aditya Gupta', 'Alok Kumar', 'Harshit Pal', 'Adarsh Shukla', 'Monish Khan', 'Vishal Kumar', 'Aditya Roy', 'Sushil Kumar', 'Ashutosh Singh', 'Ali Khan', 'Ashish Choudhary', 'Rehan Khan', 'Mohd Talha', 'Kunal Yadav', 'Krishna Dubey', 'Rahul kumar', 'Saksham Jain', 'Sumit Yadav', 'Mohammed Sadiq', 'King Khan', 'Anil Kumar', 'Aditya Arya', 'Amit Yadav', 'Lucky Sharma', 'Rishu Raj', 'Karthik Karthik', 'Dhruv Singh', 'Pooja Sharma', 'Abdul Rahman', 'Aman Pal', 'Rajesh Yadav', 'Shiva Kumar', 'Priyanshu Singh', 'Gautam Singh', 'Mukesh Kumar', 'Srinu Srinu', 'Sachin Sachin', 'Poonam Sharma', 'Abhishek Thaku

In [32]:
# Analyze feature mentions for different user segments
high_rating_feature_mentions = Counter()
low_rating_feature_mentions = Counter()

for user in high_rating_users:
    high_rating_feature_mentions += user_patterns[user]['feature_mentions'] #type: ignore

for user in low_rating_users:
    low_rating_feature_mentions += user_patterns[user]['feature_mentions'] #type: ignore

In [33]:
print("\nFeature mentions by high rating users:")
print(high_rating_feature_mentions)
print("\nFeature mentions by low rating users:")
print(low_rating_feature_mentions)


Feature mentions by high rating users:
Counter({'account': 1433, 'teenager': 1089, 'transfer': 953, 'login': 221, 'free fire': 54})

Feature mentions by low rating users:
Counter({'account': 3904, 'login': 917, 'transfer': 835, 'teenager': 265, 'free fire': 30})


In [34]:
import pandas as pd

# Save high rating users to a CSV file
high_rating_users_df = pd.DataFrame(high_rating_users, columns=['userName'])
high_rating_users_df.to_csv('high_rating_users.csv', index=False)

# Save low rating users to a CSV file
low_rating_users_df = pd.DataFrame(low_rating_users, columns=['userName'])
low_rating_users_df.to_csv('low_rating_users.csv', index=False)

# Save feature mentions by high rating users to a CSV file
high_rating_feature_mentions_df = pd.DataFrame.from_dict(high_rating_feature_mentions, orient='index', columns=['mentions'])
high_rating_feature_mentions_df.reset_index(level=0, inplace=True)
high_rating_feature_mentions_df.rename(columns={'index': 'feature'}, inplace=True)
high_rating_feature_mentions_df.to_csv('high_rating_feature_mentions.csv', index=False)

# Save feature mentions by low rating users to a CSV file
low_rating_feature_mentions_df = pd.DataFrame.from_dict(low_rating_feature_mentions, orient='index', columns=['mentions'])
low_rating_feature_mentions_df.reset_index(level=0, inplace=True)
low_rating_feature_mentions_df.rename(columns={'index': 'feature'}, inplace=True)
low_rating_feature_mentions_df.to_csv('low_rating_feature_mentions.csv', index=False)